# import、install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers fugashi ipadic
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=f0a54706633345b26c47c3506eaea3ee8bffd8205a3a86f6a0cbe22ff3c182bd
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=d74761421e63cfb122cdcb169c858216137c5bcf6ee56b35385bf37bf6473d26
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [3]:
from typing import Dict, List, Tuple, Any
from __future__ import annotations
from functools import reduce
import operator
import pandas as pd

import sys
sys.path.append('/content/drive/MyDrive/local_cuisine_search_app/modules')

from utility import load_json_obj
from pandas_utility import read_csv_df
from pipeline import NaturalLanguageProcessing

# クラスの定義

In [40]:
class NotApp:
    """
    GUIがない料理検索用のクラス

    Attributes
    ----------
    _nlp : NaturalLanguageProcessing
        固有表現を抽出するオブジェクト
    _cuisine_info_dics_maker : CuisineInfoDictionariesMaker
        検索結果の料理の情報の辞書のリストを作成するオブジェクト
    _jp_label_dic : Dict[str, str]
        英語のラベルを日本語のラベルに変換する辞書
    """
    def __init__(
            self, model_name: str, cuisine_df_path: str,  unify_dics_path: str
    ):
        """
        コンストラクタ

        Parameters
        ----------
        model_name : str
            ファインチューニング済みモデル名
        cuisine_df_path : str
            料理のデータフレームが保存されているパス
        unify_dics_path : str
            表記ゆれ統一用辞書が保存されているパス
        """
        label_info_dics: Dict[str, str | List[str]] = {
            'AREA': {
                'jp': '都道府県/地方',
                'df_cols': ['Prefecture', 'Areas']
            },
            'TYPE': {
                'jp': '種類',
                'df_cols': ['Types']

            },
            'SZN': {
                'jp': '季節',
                'df_cols': ['Seasons']
            },
            'INGR': {
                'jp': '食材',
                'df_cols': ['Ingredients list']
            }
        }

        self._nlp = NaturalLanguageProcessing(model_name)
        self._cuisine_info_dics_maker = CuisineInfoDictionariesMaker(
            cuisine_df_path, unify_dics_path, label_info_dics
        )
        self._jp_label_dic = {
            en_label: dic['jp'] for en_label, dic in label_info_dics.items()
        }

    def search(self, classifying_text: str) -> pd.DataFrame | None:
        """
        料理の検索

        Parameters
        ----------
        classifying_text : str
            固有表現抽出対象

        Returns
        -------
        pd.DataFrame | None
            料理検索結果のデータフレーム
        """
        classified_words = self._nlp.classify(classifying_text)

        self._show_classified_words(classified_words)

        searched_cuisines_df = self._cuisine_info_dics_maker.create(classified_words)

        return searched_cuisines_df

    def _show_classified_words(
            self, classified_words: Dict[str, List[str]]
    ) -> None:
        """
        固有表現の表示

        Parameters
        ----------
        classified_words : Dict[str, List[str]]
            抽出結果の辞書
            キーが分類ラベル、バリューがそのラベルの文字列のリスト
        """
        classified_words = {
            self._jp_label_dic[en_label]: words
            for en_label, words in classified_words.items()
        }

        for label, words in classified_words.items():
            print(f'{label}\n　{"、".join(words)}')

        print()


class CuisineInfoDictionariesMaker:
    """
    料理検索結果の辞書のリスト作成用クラス

    Attributes
    ----------
    _cuisine_searcher : CuisineSearcher
        料理を検索するオブジェクト
    _word_unifier : WordUnifier
        抽出結果の表記ゆれを統一するオブジェクト
    """
    def __init__(
            self,
            cuisine_df_path: str,
            unify_dics_path: str,
            label_info_dics: Dict[str, str | List[str]]
    ):
        """
        コンストラクタ

        Parameters
        ----------
        cuisine_df_path : str
            料理のデータフレームが保存されているパス
        unify_dics_path : str
            表記ゆれ統一用辞書が保存されているパス
        label_info_dics : Dict[str, str  |  List[str]]
            固有表現のラベルとラベルに対する各種設定情報の辞書
        """
        self._cuisine_searcher = CuisineSearcher(
            cuisine_df_path, label_info_dics
        )
        self._word_unifier = WordUnifier(unify_dics_path)

    def create(
            self, classified_words: Dict[str, List[str]]
    ) -> pd.DataFrame | None:
        """
        料理検索結果の辞書の作成

        Parameters
        ----------
        classified_words : Dict[str, List[str]]
            ラベルと、そのラベルに分類された固有表現の辞書

        Returns
        -------
        pd.DataFrame | None
            料理検索結果のデータフレーム
        """
        unified_words = self._word_unifier.unify(classified_words)
        searched_cuisines_df = self._cuisine_searcher.search(unified_words)

        return searched_cuisines_df


class CuisineSearcher:
    """
    料理検索用のクラス

    Attributes
    ----------
    _df : pd.DataFrame
        料理のデータフレーム
    _label_to_col : Dict[str, List[str]]
        固有表現のラベルに対して、検索するデータフレームの列のリストの辞書
    _words_dic : Dict[str, List[str]]
        データフレームの列と、列に含まれる全ての要素の辞書
    """

    def __init__(
            self,
            cuisine_df_path: str,
            label_info_dics: Dict[str, str | List[str]]
    ):
        """
        コンストラクタ

        Parameters
        ----------
        cuisine_df_path : str
            料理のデータフレームが保存されているパス
        label_info_dics : Dict[str, str  |  List[str]]
            固有表現のラベルとラベルに対する各種設定情報の辞書
        """
        self._df = read_csv_df(cuisine_df_path)
        self._label_to_col = self._create_label_to_col(label_info_dics)
        self._words_dic = {
            col: self._find_words(col)
            for cols in self._label_to_col.values() for col in cols
        }

    def _create_label_to_col(
            self, label_info_dics: Dict[str, str | List[str]]
    ) -> Dict[str, List[str]]:
        """
        label_to_colの作成

        固有表現のラベルに対応したデータフレームの列を
        特定するための辞書を作成する

        Parameters
        ----------
        label_info_dics : Dict[str, str  |  List[str]]
            固有表現のラベルとラベルに対する各種設定情報の辞書

        Returns
        -------
        Dict[str, List[str]]
            固有表現のラベルに対して、検索するデータフレームの列のリストの辞書

        Raises
        ------
        ValueError
            label_info_dicsに、データフレームに存在しない列名が含まれている場合
        """
        label_to_col: Dict[str, List[str]] = {
            label: dic['df_cols'] for label, dic in label_info_dics.items()
        }

        df_cols = self._df.columns.tolist()
        for cols in label_to_col.values():
            for col in cols:
                if col not in df_cols:
                    raise ValueError(f'"{col}"という列名は存在しません')

        return label_to_col

    def _find_words(self, col: str) -> List[str]:
        """
        列に含まれる全要素の取得

        Parameters
        ----------
        col : str
            列名

        Returns
        -------
        List[str]
            列に含まれる全ての要素のリスト
        """
        words: List[str, List[str]] = self._df[col].value_counts().index.tolist()

        if isinstance(words[0], list):
            words_lst = words
            unique_words: List[str] = []

            for words in words_lst:
                for word in words:
                    if word not in unique_words:
                        unique_words.append(word)

            return unique_words

        return words

    def search(self, unified_words: Dict[str, List[str]]) -> pd.DataFrame | None:
        """
        料理の検索

        Parameters
        ----------
        unified_words : Dict[str, List[str]]
            表記ゆれが統一された固有表現の辞書

        Returns
        -------
        pd.DataFrame | None
            検索結果の料理の情報を持つデータフレーム
        """
        on_df_words_dic = self._create_on_df_words_dic(unified_words)

        if not on_df_words_dic:
            print('いずれの語彙もデータに存在しませんでした')

            return None

        searched_cuisines_df = self._create_searched_cuisines_df(on_df_words_dic)

        return searched_cuisines_df

    def _create_on_df_words_dic(
            self, unified_words: Dict[str, List[str]]
    ) -> Dict[str, List[str]]:
        """
        データフレームに存在する固有表現だけの辞書の作成

        Parameters
        ----------
        unified_words : Dict[str, List[str]]
            表記ゆれが統一された固有表現の辞書

        Returns
        -------
        Dict[str, List[str]]
            データフレームに存在する表記ゆれが統一された固有表現の辞書
        """
        on_df_words_dic = {col: [] for col in self._words_dic}
        not_on_df_words: List[str] = []

        for label, words in unified_words.items():
            search_cols = self._label_to_col[label]

            for word in words:
                not_on_df = True

                for col in search_cols:
                    if word in self._words_dic[col]:
                        on_df_words_dic[col].append(word)

                        not_on_df = False

                        break

                if not_on_df:
                    not_on_df_words.append(word)

        if not_on_df_words:
            CuisineSearcher._show_not_on_df_words(not_on_df_words)

        on_df_words_dic = {
            col: words for col, words in on_df_words_dic.items() if words
        }

        return on_df_words_dic

    @staticmethod
    def _show_not_on_df_words(not_on_df_words: List[str]) -> None:
        """
        データフレームに存在しなかった固有表現の表示

        Parameters
        ----------
        not_on_df_words : List[str]
            データフレームに存在しなかった固有表現のリスト
        """
        words = '、'.join(not_on_df_words)
        message = f'無効な語彙:　{words}'

        print(message)

    def _create_searched_cuisines_df(
            self, words_dic: Dict[str, List[str]]
    ) -> pd.DataFrame | None:
        """
        料理の情報を持つ辞書の作成

        Parameters
        ----------
        words_dic : Dict[str, List[str]]
            検索ワードのリストを持つ辞書

        Returns
        -------
        pd.DataFrame | None
            料理の情報を持つデータフレーム
        """
        condition_lst: List[pd.Series] = []

        for col, words in words_dic.items():
            condition = self._create_condition(col, words)
            condition_lst.append(condition)

        conditions = reduce(operator.and_, condition_lst)

        searched_cuisines_df = self._df.loc[conditions]

        if searched_cuisines_df.empty:
            print('検索条件が厳しすぎて、該当料理が見つかりませんでした')

            return None

        return searched_cuisines_df

    def _create_condition(self, col: str, words: List[str]) -> pd.Series:
        """
        検索条件の作成

        Parameters
        ----------
        col : str
            絞り込み対象列
        words : List[str]
            検索ワード

        Returns
        -------
        pd.Series
            該当料理の行がTrueになったboolのシリーズ
        """
        value_type = type(self._df.at[0, col])

        if value_type is list:
            condition = self._df[col].apply(
                lambda values: any(word in values for word in words)
            )

        else:
            conditions = [self._df[col] == word for word in words]
            condition = reduce(operator.or_, conditions)

        return condition


class WordUnifier:
    """
    表記ゆれ統一用のクラス

    Attributes
    ----------
    _not_unify_labels : List[str]
        表記ゆれ統一対象ではない固有表現のラベルのリスト
    _unify_dics : Dict[str, Dict[str, str]]
        ラベルと、そのラベルの固有表現の表記ゆれ統一用の辞書の辞書
    """
    _not_unify_labels = ['SZN']

    def __init__(self, unify_dics_path: str):
        """
        コンストラクタ

        Parameters
        ----------
        unify_dics_path : str
            表記ゆれ統一用辞書が保存されているパス
        """
        self._unify_dics: Dict[str, Dict[str, str]] = load_json_obj(unify_dics_path)

    def unify(
            self, classified_words: Dict[str, List[str]]
    ) -> Dict[str, List[str]]:
        """
        表記ゆれの統一

        Parameters
        ----------
        classified_words : Dict[str, List[str]]
            ラベルと、そのラベルに分類された固有表現の辞書

        Returns
        -------
        Dict[str, List[str]]
            表記ゆれが統一された固有表現の辞書
        """
        for label, words in classified_words.items():
            if label in self._not_unify_labels:
                continue

            unify_dic = self._unify_dics[label]

            unified_words = [
                unify_dic[word] if word in unify_dic else word for word in words
            ]

            classified_words[label] = unified_words

        return classified_words

# 料理検索用オブジェクトの作成

In [41]:
model_name = 'wolf4032/bert-japanese-token-classification-search-local-cuisine'
cuisine_df_path = '/content/drive/MyDrive/local_cuisine_search_app/data/processed_data/02_local_cuisine_dataframe/local_cuisine_dataframe.csv'
unify_dics_path = '/content/drive/MyDrive/local_cuisine_search_app/data/processed_data/01_unifying_dictionaries/unifying_dictionaries.json'

not_app = NotApp(model_name, cuisine_df_path, unify_dics_path)

# 検索

In [43]:
classifying_text = '仙豆を使った野菜料理を教えて下さい'
df = not_app.search(classifying_text)
df

食材
　仙豆
種類
　野菜料理

無効な語彙:　仙豆


,Name,Image URL,Prefecture,Areas,Types,Seasons,Ingredients,Servings,Quantities,Recipes,Detail URL,Ingredients list
8,おばく,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,山梨県,"[関東, 中部, 甲信越]","[飯料理, 野菜料理]",[通年],丸麦、米、じゃがいも、大根、金時豆、さつまいも、里芋,お茶碗20杯分,"[[丸麦, 500g], [米, 2合], [じゃがいも, 200g], [大根, 200g...","[【下準備（ねぎ味噌）】\nねぎを薄切りにする。味噌、ねぎ、花カツオ、砂糖を混ぜる。, 前日...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[麦, 米, ジャガイモ, 大根, 金時豆, さつまいも, 里芋]"
18,まなめはり,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,奈良県,"[近畿, 関西]","[飯料理, 野菜料理]",[通年],春まな漬、米、カツオ節、醤油,4個分,"[[春まな漬, 100g], [米, 400g], [水, 480g], [カツオ節, 12...","[春まな漬をさっと水洗いする。, 葉を強くしぼる。, 芯の部分はみじん切りにして醤油かポン酢...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[春まな漬, 米, カツオ節, 醤油]"
35,かきまぜ,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,三重県,[東海],"[飯料理, 野菜料理, 魚料理]",[通年],米、魚、干しいたけ、油揚げ、高野豆腐、ごぼう、人参、さやいんげん、合わせ酢、だし汁など,4人分,"[[米, 3合], [水（米の2割増）, 650ml], [さやいんげん, 55g], [人...","[米を洗って分量の水につけておいて、炊く。, シビマグロはサイコロ状に切って酢につけておく。...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[米, 魚, しいたけ, 油揚げ, 豆腐, ゴボウ, 人参, さやいんげん, 酢, 出汁]"
38,こけらずし,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,高知県,[四国],"[飯料理, 野菜料理, 魚料理]",[通年],卵、米、人参、しいたけ、魚（サバなど）など,10人分,"[[米, 1.5kg], [【酢にごし】サバ, 中1尾（500g）], [【酢にごし】柚子酢...","[米は炊く1時間前に洗って、ザルにあげ、同量の水に30分浸してから炊く。, サバは3枚におろ...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[卵, 米, 人参, しいたけ, 魚, サバ]"
48,みかんずし,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,愛媛県,[四国],"[飯料理, 野菜料理]",[通年],みかんジュース、米,10人分,"[[米, 1升], [みかん生ジュース, 約1L], [水, 約1L], [みかんの皮, 適...","[米はみかんと生ジュースと水を半々の割合で炊く。, 魚は塩をふり、酢でしめる。, れんこんは...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[みかんジュース, 米]"
...,...,...,...,...,...,...,...,...,...,...,...,...
596,ねぎぬた,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,群馬県,"[関東, 北関東]",[野菜料理],[通年],ねぎ,4人分,"[[ねぎ, 5本], [麩, 適量], [【調味料】すりごま, 大さじ1], [【調味料】砂...","[ねぎは薄く斜め切りにする。, 1を熱湯で2～3分煮る。ざるに上げ、水気を絞り冷ます。, 水...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,[ねぎ]
597,ずいきの煮もの（ずいきのにもの）,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,奈良県,"[近畿, 関西]",[野菜料理],"[夏, 通年]",ずいき、薄揚げなど,4人分,"[[ずいき, 280g], [薄揚げ, 60g], [【調味料A（だし汁）】だし汁, 200...",[ずいきは皮をむき、3cmくらいの長さに切り30分ほど水に浸けた後、たっぷりの湯でゆで、水気...,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[ずいき, 薄揚げ]"
598,ざぶ汁（ざぶじる）,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,宮崎県,[九州],[野菜料理],[通年],人参、ごぼう、じゃがいも、大根、かぼちゃ,2人分,"[[いりこ昆布出汁, 230ml], [乾しいたけ, 4g], [人参, 30g], [ごぼ...","[いりこと昆布で濃いめの出汁を取る。乾しいたけを水でもどして薄切りにする。, 人参、ごぼう、...",https://www.maff.go.jp/j/keikaku/syokubunka/k_...,"[人参, ゴボウ, ジャガイモ, 大根, カボチャ]"
600,ゴンバチの油炒め（ごんばちのあぶらいため）,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,和歌山県,"[近畿, 関西]",[野菜料理],[春],ごんばち,4人分,"[[ごんばち（いたどり）, 200g], [唐辛子, 1本], [酒, 小さじ2], [砂糖...",[ごんばちは塩抜きしたものを食べやすい大きさに（火が通りやすい大きさ）切り、唐辛子は種をとり...,https://www.maff.go.jp/j/keikaku/syokubunka/k_...,[ごんばち]
